In [1]:
import os
import sys
import pandas as pd

from whoosh.index import create_in
from whoosh.fields import Schema, TEXT, ID
from whoosh.qparser import QueryParser, OrGroup
from whoosh import scoring
from whoosh.index import open_dir
from whoosh.analysis import StemFilter, RegexTokenizer, LowercaseFilter, StopFilter

Die Stichwortsuche des Sphinx Framework soll als Baseline für die getesteten Sprachmodelle dienen. Diese ist jedoch so nicht leicht portiertbar, da sie in Javascript verfasst und auf Dependencies wie JQuery aufbaut. Weitaus einfacher ist die Implementierung einer ähnlichen Search-Enginge mit **whoosh**, die aus vier Elementen besteht:

- Stemmer (Reduktion der Wörter in Dokumenten und Anfragen auf Wortstamm)
- Herausfiltern von Stoppwörtern aus Suchanfragen
- Suche nach Stichwörtern in Dokumenten
- Ranking von Dokumenten nach Relevanz nach erfolgreichem Auffinden von Stichwörtern

#### Woosh geht dabei folgendermaßen vor:

Für eine Anzahl n Dokumente (zusammenhängender Textkörper - in unserem Fall Paragraphen) wird ein Suchindex erstellt. Dafür wird eine *Analyzer* genutzt. Dieser wandelt die Sätze / Wörter des Textkörpers in Tokens um. Hierbei kann nun spezifiziert werden, nach welchen Regeln die Wörter geparsed werden. Hier wird ein *Analyzer* genutzt, der aus einem *RegexTokenizer* besteht, der Wörter einzeln parsed und dabei Satz- und Leerzeichen ignoriert, einem *LowerCaseFilter* der alle Worte in lower case umwandelt und einem *StopFilter*, welcher eine Stopwort-Liste übernimmt, die aus der Sphinx Suche übernommen wurde. Die Stoppwörter - welche in jener Liste vorhanden sind - werden beim Indizieren ignoriert. Für jedes Dokument wird also eine Liste an Worttokens erstellt, zudem werden Position des Vorkommens jener Tokens im Text abgespeichert. Zudem wird noch ein *Stemmer* eingesetzt, der Wörter auf ihren Wortstamm reduziert.

Bei einer Suchquery werden dann die Tokens der Query im Index gesucht und Dokumente, die jene Tokens enthalten zurückgegeben. Jene Dokumente werden dann noch geranked. Hierbei nutzt whoosh per Default *Okapi BM25* (https://en.wikipedia.org/wiki/Okapi_BM25), was eine Weiterentwicklung von *TF-IDF* ist, wobei die Häufikeit eines Terms in einem bestimmten Dokument (Term-Frequency) als auch die Häufigkeit eines Suchterms in allen Dokumenten (inverse Document-Frequency) eine Rolle spielt. Höhere Scores vermitteln hierbei eine höhere Relevanz bezogen auf die Query.

### Liste deutscher Stopwörter

In [2]:
stopwords = ["aber","alle","allem","allen","aller","alles","als","also","am","an","ander","andere","anderem","anderen","anderer","anderes","anderm","andern","anderr","anders","auch","auf","aus","bei","bin","bis","bist","da","damit","dann","das","dasselbe","dazu","da\u00df","dein","deine","deinem","deinen","deiner","deines","dem","demselben","den","denn","denselben","der","derer","derselbe","derselben","des","desselben","dessen","dich","die","dies","diese","dieselbe","dieselben","diesem","diesen","dieser","dieses","dir","doch","dort","du","durch","ein","eine","einem","einen","einer","eines","einig","einige","einigem","einigen","einiger","einiges","einmal","er","es","etwas","euch","euer","eure","eurem","euren","eurer","eures","f\u00fcr","gegen","gewesen","hab","habe","haben","hat","hatte","hatten","hier","hin","hinter","ich","ihm","ihn","ihnen","ihr","ihre","ihrem","ihren","ihrer","ihres","im","in","indem","ins","ist","jede","jedem","jeden","jeder","jedes","jene","jenem","jenen","jener","jenes","jetzt","kann","kein","keine","keinem","keinen","keiner","keines","k\u00f6nnen","k\u00f6nnte","machen","man","manche","manchem","manchen","mancher","manches","mein","meine","meinem","meinen","meiner","meines","mich","mir","mit","muss","musste","nach","nicht","nichts","noch","nun","nur","ob","oder","ohne","sehr","sein","seine","seinem","seinen","seiner","seines","selbst","sich","sie","sind","so","solche","solchem","solchen","solcher","solches","soll","sollte","sondern","sonst","um","und","uns","unse","unsem","unsen","unser","unses","unter","viel","vom","von","vor","war","waren","warst","was","weg","weil","weiter","welche","welchem","welchen","welcher","welches","wenn","werde","werden","wie","wieder","will","wir","wird","wirst","wo","wollen","wollte","w\u00e4hrend","w\u00fcrde","w\u00fcrden","zu","zum","zur","zwar","zwischen","\u00fcber"];

In [3]:
len(stopwords)

231

### Bauen und Testen des Analyzers

In [64]:
custom_analyzer = RegexTokenizer() | LowercaseFilter() | StopFilter(stoplist=stopwords) | StemFilter(lang="de")
[token.text for token in ana("Traceschritte werden mit der Traceschrittanalyse analysiert.")]

['traceschritt', 'traceschrittanalys', 'analysiert']

In [65]:
def createSearchableData(documents, indexdir_path, title_col=None, path_col=None, text_col=None, ending='.html'):   
 
    '''
    Schema definition: title(name of file), path(as ID), content(indexed
    but not stored), textdata (stored text content)
    '''
    
    # create Schema for indexing documents
    schema = Schema(
        title    = TEXT(stored=True),
        path     = ID(stored=True), 
        content  = TEXT(analyzer=custom_analyzer),
        textdata = TEXT(stored=True)
    )
    
    # create dir for storing indexing results
    if not os.path.exists(indexdir_path):
        print("writing index to:", indexdir_path)
        os.mkdir(indexdir_path)
 
    # Creating a index writer to add document as per schema
    ix = create_in(indexdir_path, schema)
    writer = ix.writer()
 
    # add every document to index
    for index, row in documents.iterrows():

        path = row[path_col] if path_col else None
        title = row[title_col] if title_col else None
        text = row[text_col]

        writer.add_document(title=title, path=path, content=text, textdata=text)
    
    writer.commit()

### Erstellen des Suchindex aus Dokumenten

In [7]:
documents = catalog.load("ecu_test_doku_parsed")

2022-05-23 21:33:27,932 - kedro.io.data_catalog - INFO - Loading data from `ecu_test_doku_parsed` (CSVDataSet)...


In [8]:
documents.head(1)

,Hash,Title,Filename,Body
0,uYiNfC+uN3oa3EJut1CoRA==,EasyInsert,Bedienung/EasyInsert.html,['ECU-TEST bietet eine kontextsensitive Funkti...


In [9]:
indexdir = "../data/03_primary/ecu_test_index/"
createSearchableData(documents, indexdir, title_col='Title', path_col='Filename', text_col='Body')

### Testen der Suchfunktion und Ausgabe gefundener Resultate in DataFrame

In [66]:
def query(query_str, parser='all', topN_results=10, verbose=True):
    
    result_rows = []   

    # use same analyzer for preparing query string that has been used for indexing documents
    query_str = " ".join([token.text for token in custom_analyzer(query_str)])
    if verbose: print(f'the query is: "{query_str}"')

    ix = open_dir(indexdir) 
    with ix.searcher(weighting=scoring.BM25F) as searcher:
        if parser == 'all':
            query = QueryParser("content", ix.schema).parse(query_str)
        elif parser == 'any':
            query = QueryParser("content", ix.schema, group=OrGroup).parse(query_str)

        results = searcher.search(query)

        if len(results) > 0:
            if verbose: print(f'\nsearch yielded {len(results)} results in total')
            num_res = topN_results if topN_results <= len(results) else len(results)
            for i in range(num_res):
                result_rows.append([results[i].score, results[i]['path']])
        else:
            if verbose: print('search yielded no hits')
            return []

    # convert results to dataframe and display
    df = pd.DataFrame(result_rows)
    df.columns = ['Score', 'Path']
    
    return df

### Testquery 1

In [33]:
query("Was ist ein Traceschritt?")

the query is: "traceschritt"

search yielded 34 results in total


,Score,Path
0,8.535843,Hauptprogramm/Aktionsfenster/Testschritte-Kart...
1,7.213696,TRACE-CHECK/Handbuch/Traceanalyse-Entwurf/Asse...
2,7.060910,TRACE-CHECK/Handbuch/Traceanalyse-Entwurf/Sign...
3,6.988246,TRACE-CHECK/Handbuch/Traceanalyse-Entwurf/Abla...
4,6.903731,TRACE-CHECK/Handbuch/Traceanalyse-Entwurf/Gene...
5,6.594633,Testausfuehrung/Analyse-Jobs.html
6,6.594633,Workspace/Einstellungen/Workspace/Einstellunge...
7,6.449702,TRACE-CHECK/Handbuch/Traceanalyse-Entwurf/Gene...
8,6.038557,TRACE-CHECK/Handbuch/Traceanalyse-Entwurf/Trac...
9,5.645109,TRACE-CHECK/Handbuch/Traceanalyse-Entwurf/Konz...


### Testquery 2

In [34]:
query("Wie erstellt man eine Traceschritt Analyse?")

the query is: "erstellt traceschritt analys"

search yielded 8 results in total


,Score,Path
0,12.141172,TRACE-CHECK/Handbuch/Traceanalyse-Entwurf/Konz...
1,8.530657,TRACE-CHECK/Handbuch/Traceanalyse-Entwurf/Abla...
2,8.129059,TRACE-CHECK/Tutorial/Erste/Traceanalyse/Beispi...
3,6.557274,TRACE-CHECK/Tutorial/Erste/Traceanalyse/Beispi...
4,5.120392,TRACE-CHECK/Tutorial/Erste/Traceanalyse/Beispi...
5,4.883498,TRACE-CHECK/Tutorial/Erste/Traceanalyse/Beispi...
6,3.688930,TRACE-CHECK/Handbuch/Traceanalyse-Entwurf/Diag...
7,3.554727,TRACE-CHECK/Handbuch/Traceanalyse-Entwurf/Bere...


### Testquery 3

In [35]:
query("Wie wird ein Testfall erstellt?")

the query is: "testfall erstellt"

search yielded 28 results in total


,Score,Path
0,11.520841,Testausfuehrung/Testausfuehrung.html
1,9.453027,Hauptprogramm/Aktionsfenster/Testschritte-Kart...
2,8.576262,Testausfuehrung/Analyse-Jobs.html
3,7.498254,Getting/Started/GettingStarted.html
4,6.918499,TRACE-CHECK/Handbuch/Hinweise.html
5,5.930701,Getting/Started/GettingStarted.html
6,5.658978,Einfuehrung/Testen.html
7,5.623973,Einfuehrung/Workspace/Erstellen.html
8,5.500788,Tutorials/Tutorial/RealtimeTesting.html
9,5.238597,Hauptprogramm/Editor/Maskeneditor.html


### Testquery 4

In [36]:
query("Was ist der Unterschied zwischen Testfall und Traceschritt?", parser='all')

the query is: "unterschied testfall traceschritt"
search yielded no hits


[]

In [37]:
query("Was ist der Unterschied zwischen Testfall und Traceschritt?", parser='any')

the query is: "unterschied testfall traceschritt"

search yielded 213 results in total


,Score,Path
0,10.033405,Testausfuehrung/Analyse-Jobs.html
1,8.946212,Einfuehrung/Grundlagen/Aufbau.html
2,8.535843,Hauptprogramm/Aktionsfenster/Testschritte-Kart...
3,8.283153,Testausfuehrung/Analyse-Jobs.html
4,8.283153,Tutorials/Tutorial/InteractiveTesting.html
5,8.189568,TRACE-CHECK/Handbuch/Traceanalyse-Entwurf/Sign...
6,7.755496,/build/Tools/Software/Schnittstellen/INCA/APPL...
7,7.755496,/build/Tools/Software/Schnittstellen/INCA/APPL...
8,7.553762,Einfuehrung/Tool/Anbinden.html
9,7.213696,TRACE-CHECK/Handbuch/Traceanalyse-Entwurf/Asse...


## Testen der Suchfunktion an Germanquad Datensatz

### Laden des Datensatzes als DataFrame

In [38]:
germanquad = catalog.load("germanquad_validation")

2022-05-23 21:43:06,233 - kedro.io.data_catalog - INFO - Loading data from `germanquad_validation` (CSVDataSet)...


In [39]:
germanquad.head()

,id,context,question,answers
0,40369,Aufzugsanlage\n\n=== Seilloser Aufzug ===\nAn ...,Was kann den Verschleiß des seillosen Aufzuges...,{'text': array(['elektromagnetischer Linearfüh...
1,40370,Aufzugsanlage\n\n=== Seilloser Aufzug ===\nAn ...,In welcher deutschen Stadt wird der seillose A...,"{'text': array(['Rottweil', 'Rottweil'], dtype..."
2,40366,Aufzugsanlage\n\n=== Seilloser Aufzug ===\nAn ...,Wo wurde ein seilloser Aufzug entwickelt?,{'text': array(['An der RWTH Aachen im Institu...
3,40367,Aufzugsanlage\n\n=== Seilloser Aufzug ===\nAn ...,Wie funktioniert ein seilloser Aufzug?,{'text': array(['Die Kabine wird hierbei durch...
4,40368,Aufzugsanlage\n\n=== Seilloser Aufzug ===\nAn ...,Wann muss man die Zieletage in seillosen Aufzü...,{'text': array(['vor Fahrtantritt (d.\xa0h. no...


### Hashen aller Kontexte, da Kontexte mehrfach auftreten (mehrere Fragen pro Kontext)

In [40]:
import hashlib
import base64

def hash_context(text):
    return base64.b64encode(hashlib.md5(text.encode('utf-8')).digest()).decode()

In [41]:
_test_context = germanquad['context'].iloc[1]
_test_context[:100]

'Aufzugsanlage\n\n=== Seilloser Aufzug ===\nAn der RWTH Aachen im Institut für Elektrische Maschinen wur'

In [42]:
hsh = hashlib.md5(_test_context.encode('utf-8')).digest()
base64.b64encode(hsh).decode()

'1Hu25PJK/VZwj72Is7G8vw=='

In [43]:
hash_context(_test_context)

'1Hu25PJK/VZwj72Is7G8vw=='

### Erstellen eines Index für Germanquad
Dabei werden nur unique Kontexte indiziert. Der Inhalt (Text) wird gehashed und dient als Pfad um später beim Testen einen Fragekontext einem Kontext im Index zuordnen zu können.

In [44]:
germanquad_contexts = germanquad['context']

In [45]:
germanquad_contexts.unique().shape

(474,)

In [46]:
unique_contexts_with_hash = []
for c in germanquad_contexts.unique():
    hsh = hash_context(c)
    unique_contexts_with_hash.append({"hash": hsh, "context": c})

In [47]:
pd.DataFrame(unique_contexts_with_hash).head(5)

,hash,context
0,1Hu25PJK/VZwj72Is7G8vw==,Aufzugsanlage\n\n=== Seilloser Aufzug ===\nAn ...
1,60+PszMaTC1g1qvjxof4ng==,Sichuan\n\n=== Landwirtschaft ===\nSichuan gil...
2,BNjqad/eGjINJp6eKBXS1g==,Antenne\n\n==== Polarisation ====\nAntennen st...
3,pfO/sDJuLD9rxUNBLlzYVQ==,Softwaretest\n\n== Ziele ==\n''Globales Ziel''...
4,FN6vYWPIl+LT4/A3IfIEqg==,Tuberkulose\n\n=== 19. Jahrhundert ===\nAufgru...


In [48]:
indexdir = "../data/03_primary/germanquad_index/"
createSearchableData(pd.DataFrame(unique_contexts_with_hash), indexdir, path_col='hash', text_col='context')

### Zufällige Frage wählen und prüfen ob richtiger Kontext von Stichwortsuche gefunden wird

In [62]:
# pick random sample and query the question
s = germanquad.sample()

s_q = s['question'].iloc[0]
s_id = s['id'].iloc[0]
print("question is :", s_q)

# calculate results
result = query(s_q)

# check wether hash of context - question pair is within top 5 answers
correct_context = hash_context(s['context'].iloc[0])
if len(result) > 0:
    print(f'''context is: {correct_context} and is 
        {'not ' if correct_context not in list(result['Path'])[:5] else ''}among top 5 answers''')
    
display(result)

question is : War die Republik der Vereinigten Niederlande ein Zentralstaat?
the query is: "republ vereinigt niederland zentralstaat"

search yielded 1 results in total
context is: b5lnyL2wk0wWaKesnNAtLg== and is 
        among top 5 answers


,Score,Path
0,20.34054,b5lnyL2wk0wWaKesnNAtLg==


### Berechne Top5-Score für alle Fragen + Kontexte
Frage gilt dann als richtig beantwortet, wenn Kontext zur Frage innerhalb der ersten 5 Suchergebnisse ist. Dabei müssen **alle** Tokens der Query (nach Filtern der Suchanfrage) im Kontext enthalten sein (Suchtyp: all)

In [50]:
hit_count = 0
for index, row in germanquad.iterrows():
    _q = row['question']
    _id = row['id']
    _res = query(_q, verbose=False, parser='all')
    _corr_ctxt = hash_context(row['context'])
    if len(_res) == 0: continue
    if _corr_ctxt in list(_res['Path'])[:5]:
        hit_count += 1      

In [51]:
print('accuracy is:', round(hit_count / len(germanquad), 2))

accuracy is: 0.17


### Berechnung Top5-Score mit Suchtyp: 'any'
Anstelle von allen Wörtern (Wortstämmen) der Suchanfragen muss jetzt nur **eins** der Wörter im Kontext enthalten sein, um gefunden zu werden. 

In [52]:
hit_count = 0
for index, row in germanquad.iterrows():
    _q = row['question']
    _id = row['id']
    _res = query(_q, verbose=False, parser='any')
    _corr_ctxt = hash_context(row['context'])
    if len(_res) == 0: continue
    if _corr_ctxt in list(_res['Path'])[:5]:
        hit_count += 1      

In [53]:
print('accuracy is:', round(hit_count / len(germanquad), 2))

accuracy is: 0.96


### Berechnung Top1-Score mit Suchtyp: 'any'

In [54]:
hit_count = 0
for index, row in germanquad.iterrows():
    _q = row['question']
    _id = row['id']
    _res = query(_q, verbose=False, parser='any')
    _corr_ctxt = hash_context(row['context'])
    if len(_res) == 0: continue
    if _corr_ctxt in list(_res['Path'])[:1]:
        hit_count += 1      

In [55]:
print('accuracy is:', round(hit_count / len(germanquad), 2))

accuracy is: 0.87
